In [1]:
import os, sys

sys.path.append('/Users/gabrielsucich/Desktop/option_trading/')
root = '/Users/gabrielsucich/Desktop/option_trading/'

In [2]:
from datetime import date
import pandas as pd
import numpy as np
import random

In [3]:
import Utils.jsonHelper as jh
tracked_stocks_JSON = jh.readJSON(root + "stocks/stockJSON/tracked_stocks.json")
tracked_stocks = list(tracked_stocks_JSON.keys())

In [4]:
from Models.Stocks.StockHistory import *
from Models.MetricLoggers.InterdayMetrics import *
from Models.EventLoggers.InterdayChanges import *

Logged In


In [5]:
nonconsecutive_granularity = "quartile"
nonconsecutive_volumeRecordLength = 2
nonconsecutive_pressureRecordLength = 3
nonconsecutive_priceRecordLength = 3
nonconsecutive_interdayRecordLength = 4

In [6]:
consecutive_granularity = "quartile"
consecutive_volumeRecordLength = 3
consecutive_pressureRecordLength = 3
consecutive_priceRecordLength = 3
consecutive_interdayRecordLength = 4

In [7]:
def create_stock_data(symbol, granularity, volumeRecordLength, pressureRecordLength, priceRecordLength, interdayRecordLength):
    
    try:
        stock_history_object = StockHistory(symbol, granularity = granularity)
        stock_interday_changes = InterdayChanges(stock_history_object)
        stock_interday_metrics = InterdayMetrics(stock_history_object, volumeRecordLength, pressureRecordLength, priceRecordLength, interdayRecordLength)


        stock_data = pd.DataFrame()

        stock_data["Date"] = list(stock_history_object.getHistory().keys())[max([volumeRecordLength, pressureRecordLength, priceRecordLength, interdayRecordLength]) : -1]
        stock_data["Symbol"] = symbol

        stock_volume_gradient = [stock_interday_metrics.lastVolumeMetric(date)["gradient"] for date in stock_data["Date"]]
        stock_volume_concavity = [stock_interday_metrics.lastVolumeMetric(date)["concavity"] for date in stock_data["Date"]]

        stock_price_gradient = [stock_interday_metrics.lastPriceMetric(date)["gradient"] for date in stock_data["Date"]]
        stock_price_concavity = [stock_interday_metrics.lastPriceMetric(date)["concavity"] for date in stock_data["Date"]]

        stock_pressure_mean = [stock_interday_metrics.lastPressureMetric(date)["mean"] for date in stock_data["Date"]]
        stock_pressure_gradient = [stock_interday_metrics.lastPressureMetric(date)["gradient"] for date in stock_data["Date"]]
        stock_pressure_concavity = [stock_interday_metrics.lastPressureMetric(date)["concavity"] for date in stock_data["Date"]]

        stock_interday_mean = [stock_interday_metrics.interdayMetric(date)["mean"] for date in stock_data["Date"]]
        stock_interday_gradient = [stock_interday_metrics.interdayMetric(date)["gradient"] for date in stock_data["Date"]]
        stock_interday_concavity = [stock_interday_metrics.interdayMetric(date)["concavity"] for date in stock_data["Date"]]
        stock_interday_consecutiveDrops = [stock_interday_metrics.interdayMetric(date)["consecutiveDecreases"] for date in stock_data["Date"]]
        stock_interday_consecutiveJumps = [stock_interday_metrics.interdayMetric(date)["consecutiveIncreases"] for date in stock_data["Date"]]

        stock_data["Volume Gradient"] = stock_volume_gradient
        stock_data["Volume Concavity"] = stock_volume_concavity
        stock_data["Price Gradient"] = stock_price_gradient
        stock_data["Price Concavity"] = stock_price_concavity
        stock_data["Buy Pressure Mean"] = stock_pressure_mean
        stock_data["Buy Pressure Gradient"] = stock_pressure_gradient
        stock_data["Buy Pressure Concavity"] = stock_pressure_concavity
        stock_data["Interday Mean"] = stock_interday_mean
        stock_data["Interday Gradient"] = stock_interday_gradient
        stock_data["Interday Concavity"] = stock_interday_concavity
        stock_data["Consecutive Interday Drops"] = stock_interday_consecutiveDrops
        stock_data["Consecutive Interday Jumps"] = stock_interday_consecutiveJumps

        interday_changes = stock_interday_changes.getInterdayChanges() 

        stock_data["Prev Day Change (%)"] = [interday_changes[date]["prev_day_change"] for date in stock_data["Date"]]
        stock_data["Next Day Change (%)"] = [interday_changes[date]["next_day_change"] for date in stock_data["Date"]]

        return stock_data

    except:
        print("There was an issue with {}".format(symbol))

In [8]:
def stock_data(symbols, granularity, volumeRecordLength, pressureRecordLength, priceRecordLength, interdayRecordLength):
    
        data = create_stock_data(symbols[0], granularity, volumeRecordLength, pressureRecordLength, priceRecordLength, interdayRecordLength)
        
        for symbol in symbols[1:]:
            
            data = data.append(create_stock_data(symbol, granularity, volumeRecordLength, pressureRecordLength, priceRecordLength, interdayRecordLength))
            
        
        return data

In [9]:
def update_consecutive_data():
    
    data = stock_data(tracked_stocks, consecutive_granularity, consecutive_volumeRecordLength, consecutive_pressureRecordLength, consecutive_priceRecordLength, consecutive_interdayRecordLength)
    data.to_csv("OptimalTrainingDataConsecutive.csv")
    print("Consecutive_data_updated")

def update_nonconsecutive_data():
    
    data = stock_data(tracked_stocks, nonconsecutive_granularity, nonconsecutive_volumeRecordLength, nonconsecutive_pressureRecordLength, nonconsecutive_priceRecordLength, nonconsecutive_interdayRecordLength)
    data.to_csv("OptimalTrainingDataNonconsecutive.csv")
    print("Nonconsecutive_data_updated")
    
def update_training_data():
    
    update_consecutive_data()
    update_nonconsecutive_data()
    
    

In [10]:
update_training_data()

Consecutive_data_updated
Nonconsecutive_data_updated
